In [1]:
import torch
import numpy as np

### Load Dataset

In [6]:
from torch.utils.data import DataLoader

In [3]:
data = np.load('./dataset/data.npy')

In [4]:
data.shape

(3376, 3, 96, 96)

In [7]:
dataset = DataLoader(data,batch_size = 32,shuffle=True)

## DCGAN

In [8]:
import torch.nn as nn

In [13]:
# modeled after tensorflow implementation: https://github.com/carpedm20/DCGAN-tensorflow/blob/master/model.py
class Discriminator(nn.Module):
    
    def __init__(self, init_output = 32):
        super(Discriminator,self).__init__()
        params = {'kernel_size': 5, 'stride': 2, 'padding': 2}
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = init_output, **params),
            nn.BatchNorm2d(num_features = 16),
            nn.LeakyReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels = init_output, out_channels = init_output*2, **params),
            nn.BatchNorm2d(num_features = init_output*2),
            nn.LeakyReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels = init_output*2, out_channels = init_output*4, **params),
            nn.BatchNorm2d(num_features = init_output*4),
            nn.LeakyReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels = init_output*4, out_channels = init_output*8, **params),
            nn.BatchNorm2d(num_features = init_output*8),
            nn.LeakyReLU()
        )
        self.output = nn.Sequential(
            nn.Linear(init_output*8,out_features = 1),
            nn.Sigmoid()
        )
        self.layers = [self.layer1,self.layer2,self.layer3,self.layer4,self.output]
        
    def forward(self,X):
        for layer in self.layers:
            X = layer(X)
        return X

In [ ]:
class Generator(nn.Module):
    
    def __init__(self,total_out = 4*4*1024):
        super(Generator,self).__init__()
        
        # don't forget to reshape after this
        self.layer1 = nn.Sequential(
            nn.Linear(100,out_features = total_out)
        )
        
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 1024,out_channels = 512)
        )
        